# Programming with Python

## Lecture 10: Concurrency 2

### Armen Gabrielyan

#### Yerevan State University / ASDS

#### 26 Apr, 2025

This section is heavily influenced by the following:

*References:*

- Fluent Python, Luciano Ramalho

### Race condition

A **race condition** happens when two or more threads access shared data at the same time, and the result depends on the order of execution — which is not predictable.

Think of two people writing on the same paper at the same time without coordinating. You could end up with gibberish.

It's a problem because threads may:

- Read stale or incorrect values
- Overwrite each other’s work
- Cause inconsistent or unexpected results

**See practical example 1**.

### Synchronization

**Synchronization** is the key to managing shared resources in multi-threading. It is a concept that specifies various mechanisms to ensure that no more than one concurrent thread/process can process and execute a particular program portion at a time; this portion is known as the **critical section**. Synchronization ensures that only one thread at a time can access critical sections of code or shared data, preventing race conditions and inconsistent results.

In a given program, when a thread is accessing/executing the critical section of the program, the other threads have to wait until that thread finishes executing. The typical goal of thread synchronization is to avoid any potential data discrepancies / race conditions when multiple threads access their shared resources, **allowing only one thread to execute the critical section of the program at a time** guarantees that no data conflicts occur in multithreaded applications.

Let's discuss some of the synchronization mechanisms.

#### 1. Lock / mutual exclusion (mutex)

One of the most common ways to apply thread synchronization is through the implementation of a locking mechanism. In the `threading` module, the `threading.Lock` class provides a simple and intuitive approach to creating and working with locks. Its main usage includes the following methods: 

- `threading.Lock()`: This method initializes and returns a new lock object.
- `acquire(blocking)`: When this method is called, all of the threads will run synchronously (that is, only one thread can execute the critical section at a time). The optional argument blocking allows us to specify whether the current thread should wait to acquire the lock:
  - When `blocking = 0`, the current thread does not wait for the lock and simply returns 0 if the lock cannot be acquired by the thread, or 1 otherwise
  - When `blocking = 1` (default value), the current thread blocks and waits for the lock to be released and acquires it afterwards
- `release()`: When this method is called, the lock is released.

Common pattern:

```python
import threading

lock = threading.Lock()

try:
    # Acquire the lock
    lock.acquire()
    
    # Critical section - only one thread at a time can execute this code
    critical_section_code()
finally:
    # Always release the lock, even if an exception occurs
    lock.release()
```

Lock implements context manager protocol, so it is better practice to use `with` statement:

```python
import threading

lock = threading.Lock()

with lock: # Automatically acquires and releases the lock 
    critical_region_code() # Critical section - only one thread at a time can execute this code
```

**See practical example 2**.

#### 2. Semaphore

This is one of the oldest synchronization primitives in the history of computer science, invented by the early Dutch computer scientist Edsger W. Dijkstra.

A semaphore manages an internal counter which is decremented by each `acquire()` call and incremented by each `release()` call. The counter can never go below zero; when `acquire()` finds that it is zero, it blocks, waiting until some other thread calls `release()`.

Use cases:

- **Resource pool**: Allow only 5 simultaneous DB connections
- **Rate limiting**: Max 2 API calls at once
- **Thread-safe batching**: Limit how many threads can download files at once

The `threading` module provides `threading.Semaphore` class for managing semaphores.

**See practical example 3**.

#### 3. Event

Used to signal between threads — one thread waits, another signals.

An event object manages an internal flag that can be set to true with the `set()` method and reset to false with the `clear()` method. The `wait()` method blocks until the flag is true.

The `threading` module provides `threading.Event`.

**See practical example 4**.

#### 4. Queue

A concept in computer science that is widely used in concurrent programming is queuing. **Queue** is a data structure that is a collection of different elements. Elements can be added to the end of the queue which is called enqueuing. Elements can be removed from the beginning of the queue, called dequeuing. It works in First in First out (FIFO) manner, meaning that first entered element is removed first. 

<div style="text-align: center;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/Data_Queue.svg/1200px-Data_Queue.svg.png" alt="Index" width="400" height="400"/>
</div>


The `queue` module in Python provides a simple implementation of the queue data structure. Each queue in the `queue.Queue` class can hold a specific amount of elements, and can have the following methods as its high-level API:
- `get()`: This method returns the next element of the calling queue object and removes it from the queue object
- `put()`: This method adds a new element to the calling queue object 
- `qsize()`: This method returns the number of current elements in the calling queue object (that is, its size)
- `empty()`: This method returns a Boolean, indicating whether the calling queue object is empty
- `full()`: This method returns a Boolean, indicating whether the calling queue object is full

Sometimes it is undesirable to have as many threads as the tasks we have to process. Say we have a large number of tasks to be processed, then it will be quite inefficient to spawn the same large number of threads and have each thread execute only one task. It could be more beneficial to have a **fixed number of threads (commonly known as a thread pool)** that would work through the tasks in a cooperative manner.

Here is when the concept of a queue comes in. We can design a structure in which the pool of threads will not hold any information regarding the tasks they should each execute, instead the tasks are stored in a queue (in other words task queue), and the items in the queue will be fed to individual members of the thread pool. As a given task is completed by a member of the thread pool, if the task queue still contains elements to be processed, then the next element in the queue will be sent to the thread that just became available.

**See practical example 5**.

## CPU-intensive task

In Python, CPU-intensive tasks are best handled with `multiprocessing` rather than `threading`, mainly due to the limitations of the Global Interpreter Lock (GIL).

The GIL ensures that only one thread executes Python bytecode at a time, even on multi-core processors. This means that:

- Threading does not provide real parallelism for CPU-bound tasks.
- Threads still take turns using the CPU, resulting in limited performance gain or even overhead from context switching.
- In contrast, multiprocessing creates separate processes, each with its own Python interpreter and memory space, allowing for true parallel execution across multiple CPU cores.

Key points

- Multiprocessing bypasses the GIL, enabling full CPU core usage.
- Threading is limited by the GIL for CPU-bound work.
- Multiprocessing is ideal for tasks like number crunching, image processing, or simulations.
- Threading is better suited for I/O-bound tasks (e.g., file reads, network requests).

In summary, due to the GIL, `threading` is ineffective for CPU-heavy workloads, whereas `multiprocessing` provides actual parallelism and improved performance.

Let's see this in action.

## Prime number checking

### Sequential

**See practical example 6**.

### Multi-threading

**See practical example 7**.

### Multi-processing

**See practical example 8**.

## Real-world example: simple web scraper

To demonstrate the difference in execution time between sequential and multithreaded approaches, we'll simulate downloading content from multiple URLs.

Requesting a content over a network is I/O-bound task and well-suited for multi-threading.

In [ ]:
!pip install httpx

### Sequential

**See practical example 9**.

### Multi-threading

**See practical example 10**.

### Multi-processing

We can do it with multi-processing, but since this is a I/O-bound task, it is better to solve the problem with multi-threading. Multi-processing can create additional overhead.

## Concurrent Executors and Futures

Python's `concurrent.futures` module provides a high-level interface for asynchronously executing tasks using threads or processes. It combines clean syntax with powerful concurrent programming capabilities.

### Key Components

#### Executors

The `concurrent.futures` module provides two primary executor classes:

1. `ThreadPoolExecutor`: Uses threads for concurrent execution

```python
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=4) as executor:
    # Submit tasks to be executed concurrently
```

2. `ProcessPoolExecutor`: Uses processes for concurrent execution

```python
from concurrent.futures import ProcessPoolExecutor

with ProcessPoolExecutor(max_workers=4) as executor:
    # Submit tasks to be executed concurrently
```

#### Futures

A `Future` represents the result of an asynchronous computation. It's a placeholder for a value that will be available when the computation completes.


```python
future = executor.submit(function, arg1, arg2)  # Returns immediately

# Check if done without blocking
if future.done():
    print("Task completed")

# Get result (will block until ready)
result = future.result()
```